In [9]:
import os
import json
import pymysql
import csv

# 資料庫連線
connection = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'P@ssw0rd',
    database = 'moviedb',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
)

cursor = connection.cursor()

def saveDB():
    try:
        for x in range(1,17):
            with open('{}.json'.format(x), 'r', encoding="utf-8") as fp:
                strJSON = fp.read()
                listResult = json.loads(strJSON)
                fp.close() 

            # query 模板
            sql = "INSERT INTO `moviedata` \
                    (`movie_id`, `movie_title`, `belongs_to_collection`, \
                    `movie_popularity`, `movie_releasedate`, \
                    `movie_runtime`, `movie_budget`, \
                    `movie_revenue`, `movie_imdb_id`) \
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

            for itm in listResult:
                try:
                # belongs to collection (boolean)
                    btc = itm['belongs_to_collection'] is not None
                    cursor.execute(sql, (
                            itm['id'],
                            itm['original_title'],
                            btc,
                            itm['popularity'],
                            itm['release_date'],
                            itm['runtime'],
                            itm['budget'],
                            itm['revenue'],
                            itm['imdb_id']
                            ))
                except Exception as e:
                    pass
                    continue

            sql = "INSERT INTO `actor` \
                    (`movie_id`, `actor_id`, `actor_name`, `actor_gender`, `actor_character`, `popularity`) \
                    VALUES (%s, %s, %s, %s, %s, %s)"

            for itm in listResult:
                try:
                    list1=itm['credits']['cast']
                    for itm1 in list1: 
                        cursor.execute(sql, (
                            itm['id'],
                            itm1['id'],
                            itm1['original_name'],
                            itm1['gender'],
                            itm1['character'],
                            itm1['popularity']
                            ))
                except Exception as e:
                    pass
                    continue

            sql = "INSERT INTO `productioncompany` \
                    (`movie_id`, `pc_id`, `movie_title`, `pc_name`, `pc_country`) \
                    VALUES (%s, %s, %s, %s, %s)"

            for itm in listResult:
                try:
                    list1=itm['production_companies']
                    for itm1 in list1:
                        cursor.execute(sql, (
                                itm['id'],
                                itm1['id'],
                                itm['original_title'],
                                itm1['name'],
                                itm1['origin_country']
                                ))
                except Exception as e:
                    pass
                    continue

            sql = "INSERT INTO `genres` \
                    (`movie_id`, `genre`) \
                    VALUES (%s, %s)"

            for itm in listResult:
                try:
                    list1=itm['genres']
                    for itm1 in list1:
                        cursor.execute(sql, (
                                itm['id'],
                                itm1['name']
                                ))
                except Exception as e:
                    pass
                    continue
            sql = "INSERT INTO `crew` \
                    (`movie_id`, `crew_id`, `crew_name`, `known_for_department`, `department`, `job`) \
                    VALUES (%s, %s, %s, %s, %s, %s)"

            for itm in listResult:
                try:
                    list1=itm['credits']['crew']
                    for itm1 in list1: 
                        cursor.execute(sql, (
                            itm['id'],
                            itm1['id'],
                            itm1['name'],
                            itm1['known_for_department'],
                            itm1['department'],
                            itm1['job']
                            ))  
                except Exception as e:
                    pass
                    continue

            connection.commit()

    except Exception as b:
        # 回滾
        connection.rollback()
        print("SQL 執行失敗")
        print(b)

if __name__ == "__main__":
    saveDB()